In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from review_crawler import review_crawler
import re
import lxml

In [2]:
#2020년
url_list=[
    #2020년
    #['https://tickets.interpark.com/goods/P0001849',0],
    #['https://tickets.interpark.com/goods/19017827',0],
    #['https://tickets.interpark.com/goods/20005030',0],
    #['https://tickets.interpark.com/goods/20003772',0],
    #['https://tickets.interpark.com/goods/20003508',0],
    #['https://tickets.interpark.com/goods/19011808',0],
    #['https://tickets.interpark.com/goods/20005886',0],
    #['https://tickets.interpark.com/goods/19013796',2],
    #['https://tickets.interpark.com/goods/20007209',0],
    #['https://tickets.interpark.com/goods/20010952',0],
    #['https://tickets.interpark.com/goods/20007338',0],
    #['https://tickets.interpark.com/goods/20003592',0],
    #['https://tickets.interpark.com/goods/19018778',1],
    #['https://tickets.interpark.com/goods/19016582',1],
    #['https://tickets.interpark.com/goods/20004934',0],
    #['https://tickets.interpark.com/goods/20007266',0],
    #['https://tickets.interpark.com/goods/19015390',4],
    #['https://tickets.interpark.com/goods/20003150',1],
    #['https://tickets.interpark.com/goods/19019170',1],
    #['https://tickets.interpark.com/goods/20006092',1],
    #['https://tickets.interpark.com/goods/20004265',1],
    #['https://tickets.interpark.com/goods/20005095',0],
    #['https://tickets.interpark.com/goods/20003788',4],
    #['https://tickets.interpark.com/goods/20002836',1],
    #['https://tickets.interpark.com/goods/20004236',0],
    #['https://tickets.interpark.com/goods/20003987',1],
    #['https://tickets.interpark.com/goods/20005173',0],
    #['https://tickets.interpark.com/goods/20005205',0],
    #['https://tickets.interpark.com/goods/20003711',0],
    #['https://tickets.interpark.com/goods/20003523',1],
    #['https://tickets.interpark.com/goods/20002414',0],
    #['https://tickets.interpark.com/goods/19015394',0],
    #['https://tickets.interpark.com/goods/20007956',0],
    #['https://tickets.interpark.com/goods/20007979',0],
    #['https://tickets.interpark.com/goods/20007576',0],
    #['https://tickets.interpark.com/goods/P0002368',0],
    #['https://tickets.interpark.com/goods/19018628',0],
    #['https://tickets.interpark.com/goods/20004572',1],
    #['https://tickets.interpark.com/goods/19014598',0],
    ['https://tickets.interpark.com/goods/20003530',1],
    #['https://tickets.interpark.com/goods/19011175',1],
    #['https://tickets.interpark.com/goods/20007700',0],
    #['https://tickets.interpark.com/goods/19014681',1],
    #['https://tickets.interpark.com/goods/20005654',0],
    #['https://tickets.interpark.com/goods/20004090',0],
    ['https://tickets.interpark.com/goods/19011716',1]
]

In [3]:
for i in url_list:

    driver = webdriver.Chrome()
    #해당 뮤지컬 url
    url=i[0]
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # 뮤지컬 제목
    title = soup.find('h2', attrs={'class':'prdTitle'})
    title = title.get_text().strip()
    musical_title=title


    # 예매 안내 팝업 닫기 버튼 Xpath(없으면 None으로 두기)
    closebutton = '//*[@id="popup-prdGuide"]/div/div[3]/button'

    # 리뷰 후기 버튼 Xpath
    reviewbutton_place = driver.find_element(By.XPATH,'//*[@id="productMainBody"]/nav/div/div/ul/li[3]/a').text
    time.sleep(0.5)
    reviewbutton_place=reviewbutton_place[:4]
    if reviewbutton_place=='관람후기':
        reviewbutton = '//*[@id="productMainBody"]/nav/div/div/ul/li[3]/a'
    else:
        reviewbutton = '//*[@id="productMainBody"]/nav/div/div/ul/li[4]/a'
    
    driver.quit()

    
    # 베스트 리뷰 개수
    num_best_review = i[1]
    
    if num_best_review ==0:
        # 더보기 버튼 Xpath. 
        # 첫 번째 리뷰 더보기 버튼의 Xpath 떠와서 전체에 대해 for문 이용해서 format 처리할 수 있게 li 옆 부분을 꼭!!! {}로 채워준다.
        more_xpath = '//*[@id="prdReview"]/div/div[3]/ul/li[{}]/div/div[2]/div[1]/a'
        #//*[@id="prdReview"]/div/div[3]/ul/li[{}]/div/div[2]/div[1]/a

        # 다음 페이지 Xpath.
        # 그냥 1페이지 기준에서 2페이지 버튼 Xpath 떠오고 위랑 똑같이 li 옆부분을 {}로 채워주면 된다.
        next_page_xpath = '//*[@id="prdReview"]/div/div[3]/div[2]/ol/li[{}]/a'

        # 1~10쪽에서 11~20쪽으로 넘어가는 버튼 Xpath
        ten_to_eleven_xpath = '//*[@id="prdReview"]/div/div[3]/div[2]/a'
        #//*[@id="prdReview"]/div/div[3]/div[2]/a

        # 11~20쪽에서 21~30쪽으로 넘어가는 버튼 Xpath(그 이후는 똑같으니 신경쓰지 말 것)
        next_ten_pages_xpath = '//*[@id="prdReview"]/div/div[3]/div[2]/a[2]'

        # chromedriver 로컬 저장 path. default 처리 해뒀으니 웬만하면 걍 같은 폴더에 넣자.
        # driverpath = 'C:\\어쩌구저쩌구'
    else:
        # 더보기 버튼 Xpath. 
        # 첫 번째 리뷰 더보기 버튼의 Xpath 떠와서 전체에 대해 for문 이용해서 format 처리할 수 있게 li 옆 부분을 꼭!!! {}로 채워준다.
        more_xpath = '//*[@id="prdReview"]/div/div[4]/ul/li[{}]/div/div[2]/div[1]/a'
        #//*[@id="prdReview"]/div/div[4]/ul/li[1]/div/div[2]/div[1]/a
        #//*[@id="prdReview"]/div/div[3]/ul/li[{}]/div/div[2]/div[1]/a

        # 다음 페이지 Xpath.
        # 그냥 1페이지 기준에서 2페이지 버튼 Xpath 떠오고 위랑 똑같이 li 옆부분을 {}로 채워주면 된다.
        next_page_xpath = '//*[@id="prdReview"]/div/div[4]/div[2]/ol/li[{}]/a'

        # 1~10쪽에서 11~20쪽으로 넘어가는 버튼 Xpath
        ten_to_eleven_xpath = '//*[@id="prdReview"]/div/div[4]/div[2]/a'
        #//*[@id="prdReview"]/div/div[3]/div[2]/a

        # 11~20쪽에서 21~30쪽으로 넘어가는 버튼 Xpath(그 이후는 똑같으니 신경쓰지 말 것)
        next_ten_pages_xpath = '//*[@id="prdReview"]/div/div[4]/div[2]/a[2]'


    musical_list = review_crawler(musical_title, url, closebutton, reviewbutton, num_best_review, more_xpath, next_page_xpath, ten_to_eleven_xpath, next_ten_pages_xpath)
    df = musical_list.do(1)

    title= re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", title)
    title=title.replace(" ", "")

    filepath=f"C:/Users/user/바탕 화면/엄소현/2023-1/ybigta/DA_junior_project/csv files/2020/{title}.csv"
    #파일 경로 여기만 바꾸면 돼!!!!!!
    df.to_csv(filepath,index=False, encoding="utf-8-sig")



close button found
1
2 / 307
3 / 307
4 / 307
5 / 307
6 / 307
7 / 307
8 / 307
9 / 307
10 / 307
11
12 / 307
13 / 307
14 / 307
15 / 307
16 / 307
17 / 307
18 / 307
19 / 307
20 / 307
21
22 / 307
23 / 307
24 / 307
25 / 307
26 / 307
27 / 307
28 / 307
29 / 307
30 / 307
31
32 / 307
33 / 307
34 / 307
35 / 307
36 / 307
37 / 307
38 / 307
39 / 307
40 / 307
41
42 / 307
43 / 307
44 / 307
45 / 307
46 / 307
47 / 307
48 / 307
49 / 307
50 / 307
51
52 / 307
53 / 307
54 / 307
55 / 307
56 / 307
57 / 307
58 / 307
59 / 307
60 / 307
61
62 / 307
63 / 307
64 / 307
65 / 307
66 / 307
67 / 307
68 / 307
69 / 307
70 / 307
71
72 / 307
73 / 307
74 / 307
75 / 307
76 / 307
77 / 307
78 / 307
79 / 307
80 / 307
81
82 / 307
83 / 307
84 / 307
85 / 307
86 / 307
87 / 307
88 / 307
89 / 307
90 / 307
91
92 / 307
93 / 307
94 / 307
95 / 307
96 / 307
97 / 307
98 / 307
99 / 307
100 / 307
101
102 / 307
103 / 307
104 / 307
105 / 307
106 / 307
107 / 307
108 / 307
109 / 307
110 / 307
111
112 / 307
113 / 307
114 / 307
115 / 307
116 / 307
1

: 

: 